## Step 6: Fill in the run method

Now that we have a working method that handles a single turn properly, we can build the method that handles iteration.

1. Fill in the *run* method
2. Consider the stop conditions: 
    - reaches max_iter
    - all agents happy 
    - all unhappy agents fail to become happy in one iteration
3. Change the *move* method to return a value depending on the action taken
4. Remove the intermediate output from *move* and *am_i_happy*

In [1]:
#now that we have all the logic and stuff, put it into a loop
#fill in the run method
#fix a problem found from earlier 

from numpy import random

params = {'world_size':(20,20),
          'num_agents':380,
          'same_pref' :0.4,
          'max_iter'  :100}

In [4]:
class Agent():
    def __init__(self, world, kind, same_pref):
        self.world = world
        self.kind = kind
        self.same_pref = same_pref
        self.location = None
        
    def move(self):
        #handle each agent's turn in the model iteration
        #returns 0 for happy, 1 for unhappy but moved, and 2 for unhappy and couldn't move
        happy = self.am_i_happy()

        if not happy:
            vacancies = self.world.find_vacant(return_all=True)
            for patch in vacancies:
                will_i_like_it = self.am_i_happy(loc=patch)
                if will_i_like_it is True: #check if there's somehwere I'd like
                    self.world.grid[self.location] = None #move out of current patch
                    self.location = patch                 #assign new patch to myself
                    self.world.grid[patch] = self         #update the grid
                    i_moved = True
                    # break
                    return 1 #all the 1s started out unhappy and became happy
            if not i_moved:
                return 2 #all the 2s were unhappy and still unhappy
        else:
            return 0 #all the 0 mean were happy at beginning of round.....
        #all the 0,1,2 go to a list in our def run(self) below, to determine stop conditions
    
    #took all the debug code out, could have put in an if debug statement where debug is a variable name ,but this is fine
    def am_i_happy(self, loc=False):
        #this should return a boolean for whether or not an agent is happy at a location
        #if loc is False, use current location, else use specified location
        
        if not loc:
            starting_loc = self.location
        else:
            starting_loc = loc
        
        neighbor_patches = self.world.locate_neighbors(starting_loc)
        neighbor_agents  = [self.world.grid[patch] for patch in neighbor_patches]
        neighbor_kinds   = [agent.kind for agent in neighbor_agents if agent is not None]
        num_like_me      = sum([kind == self.kind for kind in neighbor_kinds])
        
        #if an agent is in a patch with no neighbors at all, treat it as unhappy
        if len(neighbor_kinds) == 0:
            return False
        
        perc_like_me = num_like_me / len(neighbor_kinds)
        
        if perc_like_me < self.same_pref:
            return False
        else:
            return True
        

In [5]:
class World():
    def __init__(self, params):
        assert(params['world_size'][0] * params['world_size'][1] > params['num_agents']), 'Grid too small for number of agents.'
        self.params = params
        
        self.grid     = self.build_grid(  params['world_size'])
        self.agents   = self.build_agents(params['num_agents'], params['same_pref'])
        
        self.init_world()
        
    def build_grid(self, world_size):
        #create the world that the agents can move around on
        locations = [(i,j) for i in range(world_size[0]) for j in range(world_size[1])]
        return {l:None for l in locations}
    
    def build_agents(self, num_agents, same_pref):
        #generate a list of Agents that can be iterated over
        
        def _kind_picker(i):
            if i < round(num_agents / 2):
                return 'red'
            else:
                return 'blue'
            
        agents = [Agent(self, _kind_picker(i), same_pref) for i in range(num_agents)]
        random.shuffle(agents)
        return agents
    
    def init_world(self):
        #a method for all the steps necessary to create the starting point of the model
        
        for agent in self.agents:
            # while True:
            #     x = random.randint(0, self.params['world_size'][0])
            #     y = random.randint(0, self.params['world_size'][1])

            #     if self.grid[(x,y)] is None:
            #         self.grid[(x,y)] = agent
            #         agent.location = (x,y)
            #         break
            loc = self.find_vacant()
            self.grid[loc] = agent
            agent.location = loc
                    
        assert(all([agent.location is not None for agent in self.agents])), "Some agents don't have homes!"
        assert(sum([occupant is not None for occupant in self.grid.values()]) == self.params['num_agents']), 'Mismatch between number of agents and number of locations with agents.'

    def find_vacant(self, return_all=False):
        #finds all empty patches on the grid and returns a random one, unless kwarg return_all==True,
        #then it returns a list of all empty patches
        
        empties = [loc for loc, occupant in self.grid.items() if occupant is None]
        if return_all:
            return empties
        else:
            choice_index = random.choice(range(len(empties)))
            return empties[choice_index]
        
    def locate_neighbors(self, loc):
        #given a location, return a list of all the patches that count as neighbors
        include_corners = True
        
        x, y = loc
        cardinal_four = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
        if include_corners:
            corner_four = [(x+1, y+1), (x+1, y-1), (x-1, y+1), (x-1, y-1)]
            neighbors = cardinal_four + corner_four
        else:
            neighbors = cardinal_four
            
        #handle patches that are at the edges, assuming a "torus" shape
        x_max = self.params['world_size'][0] - 1
        y_max = self.params['world_size'][1] - 1
        
        def _edge_fixer(loc):
            x, y = loc
            if x < 0:
                x = x_max
            elif x > x_max:
                x = 0
                
            if y < 0:
                y = y_max
            elif y > y_max:
                y = 0
                
            return (x, y)
            
        neighbors = [_edge_fixer(loc) for loc in neighbors]        
        return neighbors
    
    
    #we want to look at 3 stopping conditions:
    #1) Max itteration
    #2) all happy if happens before max itterations
    #3) some are unhappy but there is no way to make them happy (all left unhappy are not happy at any vacant spots)
    def run(self):
        #handle the iterations of the model
        log_of_happy = [] #empty lists to put the stuff below into these, recall that lists are ordered
        log_of_moved = []
        log_of_stay  = []
        
        for iteration in range(self.params['max_iter']): #just loop over itteratoins, 1st condition
            
            random.shuffle(self.agents) #randomize agents before every iteration, we shuffle at strat or ea itteratoin to be sure no first mover biase
            move_results = [agent.move() for agent in self.agents] #calls move fn on ea agent: check if happy
            #collecting it as a list, named move_results...look at if not happy above...
            
            num_happy_at_start = sum([r==0 for r in move_results])
            num_moved          = sum([r==1 for r in move_results])
            num_stayed_unhappy = sum([r==2 for r in move_results])
            
            log_of_happy.append(num_happy_at_start)
            log_of_moved.append(num_moved) #gettting a list ex [160, 120, 79, 1, 0, 0]
            log_of_stay .append(num_stayed_unhappy)
            
            if log_of_moved[-1] == log_of_stay[-1] == 0: #second condition, nobody tried to move and happy
                print('Everyone is happy!  Stopping after iteration {}.'.format(iteration))
                break
                #could instead have checke if all happy at start, here we check if 
            elif log_of_moved[-1] == 0 and log_of_stay[-1] > 0: #third condtion, nobody moved but someone is still unhappy
                print('Some agents are unhappy, but they cannot find anywhere to move to.  Stopping after iteration {}.'.format(iteration))
                break
    
    #general note: read the traceback errors, another reason not to use Jupyter, less descriptive error codes
    
    def report(self):
        #report final results after run ends
        pass

In [6]:
world = World(params)
world.run()
#error, but only sometimes!
#so there must be a conditional doing it (only happens sometimes) 

UnboundLocalError: local variable 'i_moved' referenced before assignment

## Step 7: Add reporting information

Now that our model runs, we need a way to report and collect results.  There are many possible statistics we might collect from such a model, including dataframes of results over time, plots, or other measures.

1. Implement basic reporting in the *report* method
2. Introduce writing to file
3. Fixed a missing i_moved=False declaration in the *move* method

In [7]:
#we got an error so lets see what it was,
#and lets export

from numpy import random, mean

params = {'world_size':(20,20),
          'num_agents':380,
          'same_pref' :0.4,
          'max_iter'  :100}

In [8]:
class Agent():
    def __init__(self, world, kind, same_pref):
        self.world = world
        self.kind = kind
        self.same_pref = same_pref
        self.location = None
        
    def move(self):
        #handle each agent's turn in the model iteration
        #returns 0 for happy, 1 for unhappy but moved, and 2 for unhappy and couldn't move
        happy = self.am_i_happy()

        if not happy:
            vacancies = self.world.find_vacant(return_all=True)
            for patch in vacancies: 
                i_moved = False #problem was here, added this line so that it starts as False, otherwise if unhappy and not move didn't have anything equal to i_moves
                will_i_like_it = self.am_i_happy(loc=patch)
                if will_i_like_it is True:
                    self.world.grid[self.location] = None #move out of current patch
                    self.location = patch                 #assign new patch to myself
                    self.world.grid[patch] = self         #update the grid
                    i_moved = True #if finds spot likes
                    # break
                    return 1
            if not i_moved: #error message was pointing here, we didn't have i_moved equal anything if they didn't move
            #if i_moved is False: #this does the same thing
                return 2
            #we could have said if i_moved is False: -> T
            #or if not True: -> F
            # if not i_moved is same as if not True so -> F so this is shorthand
        else:
            return 0
        
    def am_i_happy(self, loc=False, neighbor_check=False): #added a new keward
        #this should return a boolean for whether or not an agent is happy at a location
        #if loc is False, use current location, else use specified location
        
        if not loc:
            starting_loc = self.location
        else:
            starting_loc = loc
        
        neighbor_patches = self.world.locate_neighbors(starting_loc)
        neighbor_agents  = [self.world.grid[patch] for patch in neighbor_patches]
        neighbor_kinds   = [agent.kind for agent in neighbor_agents if agent is not None]
        num_like_me      = sum([kind == self.kind for kind in neighbor_kinds])
        
        #for reporting purposes, allow checking of the current number of similar neighbors
        if neighbor_check: #I don't jsut want the summ, I want a list
            return [kind == self.kind for kind in neighbor_kinds]
        
        #if an agent is in a patch with no neighbors at all, treat it as unhappy
        if len(neighbor_kinds) == 0:
            return False
        
        perc_like_me = num_like_me / len(neighbor_kinds)
        
        if perc_like_me < self.same_pref:
            return False
        else:
            return True
        

In [9]:
class World():
    def __init__(self, params):
        assert(params['world_size'][0] * params['world_size'][1] > params['num_agents']), 'Grid too small for number of agents.'
        self.params = params
        self.reports = {}
        
        self.grid     = self.build_grid(  params['world_size'])
        self.agents   = self.build_agents(params['num_agents'], params['same_pref'])
        
        self.init_world()
        
    def build_grid(self, world_size):
        #create the world that the agents can move around on
        locations = [(i,j) for i in range(world_size[0]) for j in range(world_size[1])]
        return {l:None for l in locations}
    
    def build_agents(self, num_agents, same_pref):
        #generate a list of Agents that can be iterated over
        
        def _kind_picker(i):
            if i < round(num_agents / 2):
                return 'red'
            else:
                return 'blue'
            
        agents = [Agent(self, _kind_picker(i), same_pref) for i in range(num_agents)]
        random.shuffle(agents)
        return agents
    
    def init_world(self):
        #a method for all the steps necessary to create the starting point of the model
        
        for agent in self.agents:
            # while True: #this was an infinate loop, a way to say just keep going untill break 
                            #but this wasn't very efficient so we wrote a better way and commented out
            #     x = random.randint(0, self.params['world_size'][0])
            #     y = random.randint(0, self.params['world_size'][1])

            #     if self.grid[(x,y)] is None:
            #         self.grid[(x,y)] = agent
            #         agent.location = (x,y)
            #         break
            #ex// see this kind of thig with user input, need to keep looping until they do it right, then can break out of loop
            #or could be type 1-10 or "help", if they keep typoing wroing stuff, keep looping to aks them to try again
            loc = self.find_vacant()
            self.grid[loc] = agent
            agent.location = loc
                    
        assert(all([agent.location is not None for agent in self.agents])), "Some agents don't have homes!"
        assert(sum([occupant is not None for occupant in self.grid.values()]) == self.params['num_agents']), 'Mismatch between number of agents and number of locations with agents.'
        
        #set up some reporting dictionaries
        self.reports['integration'] = []

    def find_vacant(self, return_all=False):
        #finds all empty patches on the grid and returns a random one, unless kwarg return_all==True,
        #then it returns a list of all empty patches
        
        empties = [loc for loc, occupant in self.grid.items() if occupant is None]
        if return_all:
            return empties
        else:
            choice_index = random.choice(range(len(empties)))
            return empties[choice_index]
        
    def locate_neighbors(self, loc):
        #given a location, return a list of all the patches that count as neighbors
        include_corners = True
        
        x, y = loc
        cardinal_four = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
        if include_corners:
            corner_four = [(x+1, y+1), (x+1, y-1), (x-1, y+1), (x-1, y-1)]
            neighbors = cardinal_four + corner_four
        else:
            neighbors = cardinal_four
            
        #handle patches that are at the edges, assuming a "torus" shape
        x_max = self.params['world_size'][0] - 1
        y_max = self.params['world_size'][1] - 1
        
        def _edge_fixer(loc):
            x, y = loc
            if x < 0:
                x = x_max
            elif x > x_max:
                x = 0
                
            if y < 0:
                y = y_max
            elif y > y_max:
                y = 0
                
            return (x, y)
            
        neighbors = [_edge_fixer(loc) for loc in neighbors]        
        return neighbors
            
    def report_integration(self): #need to show level fo integration, useing avg here
        diff_neighbors = [] #gonna put a list comprehension in here
        for agent in self.agents:
            diff_neighbors.append(sum([not a for a in agent.am_i_happy(neighbor_check=True)])) #new keward back in am_i_happy: neighbur_check and had it default to False
        self.reports['integration'].append(round(mean(diff_neighbors), 2))
  
    #look at it 
   # def report_integration(self): #need to show level fo integration, useing avg here
   #     diff_neighbors = [] #list of neighbours not like each agent 
   #     for agent in self.agents:
   #         diff_neighbors.append(sum( #so total of neigh different from me
   #             [not a #this reverses a list of boulians not T -> F not F -> T, so if different from me now True
   #              for a in agent.am_i_happy(neighbor_check=True)])) #new keward back in am_i_happy: neighbur_check and had it default to False
   #     self.reports['integration'].append(round(
   #                                 mean(diff_neighbors) #takeing the average of our new list
  #                                    , 2))
    #he tracked number different you coudl do other stuff (lit review informed)
        
    def run(self):
        #handle the iterations of the model
        log_of_happy = []
        log_of_moved = []
        log_of_stay  = []
                
        self.report_integration() #starting point, no moving, start of loop. How integrated at onset
        log_of_happy.append(sum([a.am_i_happy() for a in self.agents])) #starting happiness, how many happy before anyone moves
        log_of_moved.append(0) #no one moved at startup #no one moves bc we haven't done anything yet, not itterated
        log_of_stay.append(0) #no one stayed at startup #we havent' given them a chance ot move

        for iteration in range(self.params['max_iter']):
            
            random.shuffle(self.agents) #randomize agents before every iteration
            move_results = [agent.move() for agent in self.agents]
            self.report_integration() #call this, so see new results for integration
            
            num_happy_at_start = sum([r==0 for r in move_results])
            num_moved          = sum([r==1 for r in move_results])
            num_stayed_unhappy = sum([r==2 for r in move_results])
            
            log_of_happy.append(num_happy_at_start)
            log_of_moved.append(num_moved)
            log_of_stay .append(num_stayed_unhappy)
            
            if log_of_moved[-1] == log_of_stay[-1] == 0:
                print('Everyone is happy!  Stopping after iteration {}.'.format(iteration))
                break
            elif log_of_moved[-1] == 0 and log_of_stay[-1] > 0:
                print('Some agents are unhappy, but they cannot find anywhere to move to.  Stopping after iteration {}.'.format(iteration))
                break
    
        self.reports['log_of_happy'] = log_of_happy
        self.reports['log_of_moved'] = log_of_moved
        self.reports['log_of_stay']  = log_of_stay
        
        self.report()
        
    def report(self, to_file=True): #default to true means I'm overwriting each time
        #report final results after run ends
        reports = self.reports
        print('\nAll results begin at time=0 and go in order to the end.\n')
        print('The average number of neighbors an agent has not like them:', reports['integration'])
        print('The number of happy agents:', reports['log_of_happy'])
        print('The number of moves per turn:', reports['log_of_moved'])
        print('The number of agents who failed to find a new home:', reports['log_of_stay'])
        
        #python looks for stuff in strings, ex {}
        #another is \ this meas don't look at this
        #ex:
        'i have \{\} to use {}'.format('format')'
        #so leave the {} as {} dont fill in the first one! only the secodn one
        #but that also means I can't use \
        #you can escape the escape char: \\ means don't treat the \ as a \
        #or can do / instead of \ 
        #best, just put r tells python don't evaluate anything in here
        #r means raw string 
                
        if to_file:
            out_path = r'c:\users\jeff\desktop\abm_results.csv' #the r here means raw sting
            #shold change the name, but looks like my computer did path on own. 
            with open(out_path, 'w') as f: #context management, open a file, write on it and close it
                #with open knows to close when get out of indented block 
                headers = 'turn,integration,num_happy,num_moved,num_stayed\n'
                f.write(headers)
                for i in range(len(reports['log_of_happy'])):
                    line = ','.join([str(i),
                                     str(reports['integration'][i]), 
                                     str(reports['log_of_happy'][i]),
                                     str(reports['log_of_moved'][i]),
                                     str(reports['log_of_stay'][i]),
                                     '\n'
                                     ])
                    f.write(line)
            print('\nResults written to:', out_path)

In [12]:
world = World(params)
world.run()

Everyone is happy!  Stopping after iteration 5.

All results begin at time=0 and go in order to the end.

The average number of neighbors an agent has not like them: [3.68, 1.91, 1.36, 1.15, 1.13, 1.11, 1.11]
The number of happy agents: [283, 278, 351, 362, 378, 378, 380]
The number of moves per turn: [0, 102, 29, 18, 2, 2, 0]
The number of agents who failed to find a new home: [0, 0, 0, 0, 0, 0, 0]

Results written to: c:\users\jeff\desktop\abm_results.csv


In [ ]:
world.reports
#I'm building 4 lists
#when randomly distributed: how integraetd, how many moved...
#then level of integreation afer start moving, how many moved .....
#so 4 lists each entry is an intteration
#so first element is onset, second is first itteration, ......
#no body moved at last time

#we want output though 
#see to file method above

In [ ]:
import os
os.sep #imports stuff so that know what to do for diff opperationg systems 

In [15]:
#HW, playing around


agent.kind() is red


NameError: name 'agent' is not defined

In [22]:
agent = Agent(world, kind, same_pref)
agent.kind


NameError: name 'kind' is not defined

In [23]:
num_like_me

NameError: name 'num_like_me' is not defined

In [25]:
self.kind


AttributeError: type object 'Agent' has no attribute 'self'